In [22]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt
import math

In [23]:
import numpy as np
private_labels=np.load("Nohot_private_label.npy")
private_pixels=np.load("private_pixels.npy")
public_labels=np.load("Nohot_public_label.npy")
public_pixels=np.load("public_pixels.npy")
train_labels=np.load("Nohot_train_label.npy")
train_pixels=np.load("train_pixels.npy")

In [24]:
X_train , y_train = train_pixels,train_labels
X_valid , y_valid = public_pixels,public_labels
X_test , y_test = private_pixels,private_labels

In [25]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [26]:
import torch
import torch.nn
input_data=torch.FloatTensor(X_train_centered)
label=torch.LongTensor(y_train)

In [27]:
input_data1=torch.FloatTensor(X_valid_centered)
label1=torch.LongTensor(y_valid)

In [28]:
input_data2=torch.FloatTensor(X_test_centered)
label2=torch.LongTensor(y_test)

In [29]:
input_data=input_data.transpose(1,3)
input_data1=input_data1.transpose(1,3)
input_data2=input_data2.transpose(1,3)

In [30]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
trainset = TensorDataset(input_data,label)
valset = TensorDataset(input_data1,label1)
testset = TensorDataset(input_data2,label2)

In [31]:
partition={'train':trainset,'val':valset,'test':testset}

In [32]:
import torch
import torch.nn as nn

In [33]:
class Bottleneck(nn.Module):
    def __init__(self, nChannels,growthRate):
        super(Bottleneck,self).__init__()
        interChannels = 4*growthRate
        self.bn1=nn.BatchNorm2d(nChannels)
        self.conv1=nn.Conv2d(nChannels, interChannels,kernel_size=1,
                            bias=False)
        self.bn2=nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3,
                               padding=1, bias=False)
    def forward(self,x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

In [34]:
class Transition(nn.Module): # 반으로 줄여주는 작업임.
    def __init__(self, nChannels, nOutChannels):
        super(Transition, self).__init__()
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, nOutChannels, kernel_size=1,
                               bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = F.avg_pool2d(out, 2)
        return out

In [35]:
class DenseNet(nn.Module):
    def __init__(self, growthRate, depth, reduction, nClasses, bottleneck):
        super(DenseNet, self).__init__()

        nDenseBlocks = (depth-4) // 3 # 만약 depth가 100이라면 32
        if bottleneck:
            nDenseBlocks //= 2 # Bottleneck이 있다면  16

        nChannels = 2*growthRate # growthRate 12
        self.conv1 = nn.Conv2d(1, nChannels, kernel_size=3, padding=1,
                               bias=False)
        # 1, 24, 50, 50
        
        #nDenseBlock => 16(Bottleneck)
        #nChannel => 24
        #growthRath => 12
        self.dense1 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 216, 50, 50
        # 12*18 => 12 == growthRath 
        nChannels += nDenseBlocks*growthRate
        # 24 + 192 =>nChannels
        #print(nChannels)
        nOutChannels = int(math.floor(nChannels*reduction)) #math.floor는 소수점 내림.
        # nOutChannels => 108
        self.trans1 = Transition(nChannels, nOutChannels)
        # 1, 108, 25, 25 반반으로 짜름

        nChannels = nOutChannels
        # 1, 108, 25, 25 => x 
        # nChannels => 108
        self.dense2 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 300, 25, 25
        # 108 + 12*16 = > 300
        nChannels += nDenseBlocks*growthRate
        nOutChannels = int(math.floor(nChannels*reduction))
        self.trans2 = Transition(nChannels, nOutChannels)
        # 1, 150, 12 ,12

        nChannels = nOutChannels
        self.dense3 = self._make_dense(nChannels, growthRate, nDenseBlocks, bottleneck)
        # 1, 342, 12, 12
        # 192를 계속 더해줌 Tip!
        # 150+192=>342
        nChannels += nDenseBlocks*growthRate
        # print(nChannels)
        # nChannels => 342
        

        self.bn1 = nn.BatchNorm2d(nChannels)
        self.fc = nn.Linear(nChannels, nClasses)

        # layer의 초기화 
        for m in self.modules():
            if isinstance(m, nn.Conv2d): # m이 nn.Conv2d이면
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # n=> size
                m.weight.data.normal_(0, math.sqrt(2. / n))
                #zero-mean Gaussian distribution Standard deviation
                
                #print(m.weight.data)
                # 가중치 초기화
            elif isinstance(m, nn.BatchNorm2d): # m이 nn.BatchNorm2d이면
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                # bias => 0
            elif isinstance(m, nn.Linear): # m이 nn.Linear이면
                m.bias.data.zero_()
                # bias=> 0
        # xavier method => initial differnece thesis ReLu/PReLU 제외
        # delving deep into rectifiers: surpassing human level performance on imagenet classification

    def _make_dense(self, nChannels, growthRate, nDenseBlocks, bottleneck):
        layers = []
        for i in range(int(nDenseBlocks)):
            if bottleneck:
                layers.append(Bottleneck(nChannels, growthRate))
            else:
                layers.append(SingleLayer(nChannels, growthRate))
            nChannels += growthRate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        # 1, 24, 50, 50
        out = self.trans1(self.dense1(out))
        # 1, 216, 50, 50
        # 1, 108, 25, 25
        out = self.trans2(self.dense2(out))
        # 1, 300, 25, 25
        # 1, 150, 12 ,12
        out = self.dense3(out)
        # 1, 342, 12 ,12
        out = torch.squeeze(F.avg_pool2d(F.relu(self.bn1(out)), 8))
        # avg_pool2d out => 1, 342, 1, 1
        out = F.log_softmax(self.fc(out))
        return out

In [36]:
def dimension_check():
    net = DenseNet(growthRate=12, depth=100, reduction=0.5,
                                bottleneck=True, nClasses=7)
    x= torch.randn(1,1,48,48)
    y= net(x)
    print(y.size())

In [37]:
dimension_check()

torch.Size([7])


/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:95: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [38]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=64,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [39]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=64, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [40]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=64, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [41]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [ ]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net = DenseNet(growthRate=12, depth=100, reduction=0.5,
                                bottleneck=True, nClasses=7)
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  0%|          | 1/449 [00:00<01:27,  5.12it/s]

time : 88.34, train_loss : 1.57, train_acc : 37.40,val_acc : 44.19,val_loss : 1.44


  0%|          | 1/449 [00:00<01:27,  5.10it/s]

time : 88.92, train_loss : 1.23, train_acc : 53.00,val_acc : 55.98,val_loss : 1.16


  0%|          | 1/449 [00:00<01:26,  5.17it/s]

...

  0%|          | 1/449 [00:00<01:27,  5.11it/s]

time : 88.95, train_loss : 0.12, train_acc : 95.80,val_acc : 61.55,val_loss : 2.08


  0%|          | 1/449 [00:00<01:27,  5.13it/s]

time : 88.95, train_loss : 0.11, train_acc : 96.33,val_acc : 59.01,val_loss : 2.49


 26%|██▋       | 118/449 [00:22<01:03,  5.21it/s]

In [44]:
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Dense_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Dense_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Dense_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/Dense_val_acc.npy",val_acc)

In [46]:
val_accs

[44.19,
 55.98,
 52.66,
 56.78,
 58.9,
 60.52,
 60.46,
 61.63,
 62.05,
 62.55,
 58.09,
 62.19,
 60.24,
 57.37,
 62.61,
 56.48,
 59.13,
 61.74,
 62.25,
 59.52,
 60.52,
 58.68,
 60.32,
 60.96,
 61.91,
 57.73,
 62.16,
 60.55,
 61.55,
 59.01,
 62.33,
 63.14,
 61.47,
 60.69,
 62.13,
 60.96,
 61.08,
 59.68,
 61.41,
 62.66,
 60.43,
 59.96,
 62.75,
 62.16,
 62.33,
 61.41,
 62.05,
 62.72,
 61.27,
 62.02,
 58.34,
 62.02,
 61.52,
 59.52,
 61.27,
 62.58,
 62.05,
 59.1,
 60.57,
 62.41,
 61.77,
 63.0,
 60.96,
 62.47,
 61.63,
 61.47,
 61.49,
 62.55,
 61.63,
 62.69,
 60.94,
 61.74,
 60.55,
 61.72,
 60.63,
 61.58,
 62.66,
 62.19,
 60.02,
 62.5,
 61.08,
 62.16,
 61.21,
 60.71,
 60.57,
 62.75,
 60.43,
 62.41,
 61.97,
 60.63,
 62.86,
 61.86,
 62.66,
 61.74,
 61.49,
 63.08,
 62.16,
 61.44,
 61.86,
 60.88]